In [ ]:
!pip3 install virtualenv
!virtualenv -p python3.8.10 python38

!source /content/drive/MyDrive/Colab/RL_NLP/python38/bin/activate
!python --version

In [ ]:
from google.colab import drive
from os.path import join  
import sys

def mount_drive(ROOT):
    drive.mount(ROOT, force_remount=True)

ROOT = '/content/drive'
mount_drive(ROOT)

MY_GOOGLE_DRIVE_PATH = 'MyDrive/RL_NLP' 
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
print("PROJECT_PATH: ", PROJECT_PATH)   
%cd "{PROJECT_PATH}"
sys.path.insert(0, PROJECT_PATH)

In [ ]:
!pip install -r "{PROJECT_PATH}/metadata/requirements.txt"

# RUN FROM HERE.

In [ ]:
import numpy as np 
import pandas as pd 
import time
import utils as U
import model as M
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt 
import json
import copy

In [ ]:
!pip install -e .

In [ ]:
import gym
%cd gym-examples
import gym_examples
%cd ..
import numpy as np

### Load Models & Results.

In [ ]:
# Loading Results.
episode_reward_hist=np.load('results/ep_tot.npy')
episode_avg_reward=np.load('results/ep_avg.npy')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_tr=M.NNModel().to(device)
model_tr.load_state_dict(torch.load('results/model.ml'))

In [ ]:
def plotter(arr,xlabel,ylabel,title):
    plt.plot(arr)
    plt.xlabel(f'{xlabel}')
    plt.ylabel(f'{ylabel}')
    plt.title(f'{title}')
    plt.savefig(f'results/{title}.png')
    plt.show()
    print()

plotter(episode_reward_hist,'Episodes','Reward','Reward Per Episode VS Episodes')
plotter(episode_avg_reward,'Episodes','Avg Reward','Avg Reward Per Episode VS Episodes')

### Test New Policy.

In [ ]:
def policy(observation,device):
    U.phi(observation)
    action=(M.predict(model_tr,[observation],device))[1]
    return action.item()

dbg=True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
env = gym.make('gym_examples/RlNlpWorld-v0',render_mode="rgb_array")
episodes=1
for _ in range(episodes):
    cumulative_reward,steps=0,0
    observation = env.reset(seed=42)
    cnt=0
    while True:
        action = policy(copy.deepcopy(observation),device)  # User-defined policy function
        if dbg==True:
            plt.imshow(observation['visual'])
            plt.show()
            print()
            print(f'action {action}')
        observation, reward, terminated, info = env.step(action)
        cumulative_reward+=reward
        steps+=1
        if terminated:
            break
    print(f'Cumulative Reward ~ {cumulative_reward}; TimeTaken ~ {steps}')
env.close()